# AnomalyDINO: Google Colab実行用ノートブック

このノートブックでは、AnomalyDINOをGoogle Colab（A100等）で実行できます。

## 前提条件
1. MVTec-ADデータセットをGoogle Driveにアップロード済み
2. Colab Pro/Pro+でA100を選択（推奨）

## セットアップ手順
1. ランタイム → ランタイムタイプを変更 → GPU（A100推奨）
2. 以下のセルを順番に実行

## Step 1: リポジトリのクローンと環境構築

In [ ]:
# GitHubリポジトリをクローン
!git clone https://github.com/YOUR_USERNAME/AnomalyDINO-Colab.git
%cd AnomalyDINO-Colab

# 依存関係をインストール
!pip install -r requirements.txt

# GPU確認
!nvidia-smi

## Step 2: Google Driveマウントとデータセット設定

In [ ]:
from google.colab import drive
import os

# Google Driveをマウント
drive.mount('/content/drive')

# 利用可能なファイルを確認
!ls /content/drive/MyDrive/

In [ ]:
# MVTec-ADデータセットのパスを指定（実際のパスに変更してください）
drive_data_path = "/content/drive/MyDrive/mvtec_anomaly_detection.zip"  # ZIPファイルの場合
# drive_data_path = "/content/drive/MyDrive/mvtec_anomaly_detection"  # フォルダの場合

# セットアップスクリプトを実行
from colab_setup import setup_colab_environment, download_mvtec_from_drive

# 環境セットアップ
setup_colab_environment()

# データセットダウンロード
download_mvtec_from_drive(drive_data_path)

## Step 3: 異常検知実行

### オプション1: 1-shot（論文再現、高速テスト）

In [ ]:
# 1-shot、1シードでクイックテスト（約30分）
!python run_anomalydino.py \
    --dataset MVTec \
    --data_root data/mvtec_anomaly_detection \
    --model_name dinov2_vitb14 \
    --resolution 448 \
    --shots 1 \
    --num_seeds 1 \
    --preprocess informed \
    --device cuda:0

### オプション2: 論文完全再現（3シード）

In [ ]:
# 1-shot、3シードで論文再現（約2-3時間）
!python run_anomalydino.py \
    --dataset MVTec \
    --data_root data/mvtec_anomaly_detection \
    --model_name dinov2_vitb14 \
    --resolution 448 \
    --shots 1 \
    --num_seeds 3 \
    --preprocess informed \
    --device cuda:0

## Step 4: 結果確認

In [ ]:
# 結果ファイルを表示
import json
import glob

# メトリクスファイルを検索
result_files = glob.glob("results*/metrics_seed=*.json")
print(f"結果ファイル数: {len(result_files)}")

# 各シードの結果を表示
for file in result_files:
    print(f"\n=== {file} ===")
    with open(file, 'r') as f:
        metrics = json.load(f)
        if 'mean_classification_au_roc' in metrics:
            auroc = metrics['mean_classification_au_roc'] * 100
            print(f"Mean AUROC: {auroc:.2f}%")

# 結果フォルダの内容を確認
!ls -la results*/

## Step 5: 結果のダウンロード

In [ ]:
# 結果をZIPファイルにまとめてダウンロード
!zip -r anomalydino_results.zip results*/

from google.colab import files
files.download('anomalydino_results.zip')

print("✅ 結果のダウンロード完了")
print("期待結果: 1-shot AUROC 96.6% ± 数%")